In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [2]:
datapath='F:\\deeplearning\\datasets'

In [3]:
model=nn.Sequential(nn.Conv2d(3,16,kernel_size=3,padding=1),nn.Tanh(),nn.MaxPool2d(2),
                    nn.Conv2d(16,8,kernel_size=3,padding=1),nn.Tanh(),nn.MaxPool2d(2),
                    nn.Linear(8*8*8,32),nn.Tanh(),nn.Linear(32,2))

In [4]:
numel_list = [p.numel() for p in model.parameters()] 
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.active1=nn.Tanh()
        self.pool1=nn.MaxPool2d(2)
        self.conv2=nn.Conv2d(16,8,kernel_size=3,padding=1)
        self.active2=nn.Tanh()
        self.pool2=nn.MaxPool2d(2)
        self.fullyconnected1=nn.Linear(8*8*8,32)
        self.active3=nn.Tanh()
        self.fullyconnected2=nn.Linear(32,2)
    def forward(self,x):
        input=self.pool1(self.active1(self.conv1(x)))
        input=self.pool2(self.active2(self.conv2(input)))
        input=input.view(-1,8*8*8)
        input=self.active3(self.fullyconnected1(input))
        input=self.fullyconnected2(input)
        return input
        

In [6]:
model = Net() 
numel_list = [p.numel() for p in model.parameters()] 
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [7]:
import datetime 

In [10]:
tensor_cifar10=datasets.CIFAR10(datapath,train=False,download=False,transform=transforms.ToTensor())

In [11]:
imgs=torch.stack([img for img ,_ in tensor_cifar10],dim=3)

In [12]:
a = imgs.view(3, -1).mean(dim=1)
b = imgs.view(3, -1).std(dim=1)

In [13]:
transforms_cifar10_val= datasets.CIFAR10(datapath,train=False,download=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(a, b)]))   

In [14]:
transforms_cifar10= datasets.CIFAR10(datapath,train=True,download=False,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(a, b)]))    

In [15]:
label_map={0:0,2:1}
cifar2=[ (img,label_map[label]) for img,label in transforms_cifar10 if label in [0,2] ]

In [27]:
cifar2_val=[(img,label_map[label]) for img,label in transforms_cifar10_val if label in [0,2] ]

In [16]:
len(cifar2)

10000

In [17]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)

In [28]:
val_loader = torch.utils.data.DataLoader(cifar2_val,batch_size=64,shuffle=False)

In [20]:
learningrate=1e-2
n_epochs=50

In [19]:
optimizer=optim.SGD(model.parameters(),lr=learningrate)

In [21]:
loss_fn=nn.CrossEntropyLoss()

In [22]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader): 
   for epoch in range(1, n_epochs + 1): 
      loss_train = 0.0 
      for imgs, labels in train_loader: 
        outputs = model(imgs) 
        loss = loss_fn(outputs, labels) 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        loss_train += loss.item() 
      if epoch == 1 or epoch % 10 == 0: 
        print('{} Epoch {}, Training loss {}'.format( datetime.datetime.now(), 
                                                      epoch, loss_train / len(train_loader)))

In [23]:
training_loop( 
 n_epochs = 100, 
 optimizer = optimizer, 
 model = model, 
 loss_fn = loss_fn, 
 train_loader = train_loader, 
)

2024-08-29 23:57:44.166492 Epoch 1, Training loss 0.6009923371539754
2024-08-29 23:57:56.605043 Epoch 10, Training loss 0.32996869628216813
2024-08-29 23:58:10.597173 Epoch 20, Training loss 0.29696458103550466
2024-08-29 23:58:24.241008 Epoch 30, Training loss 0.2757971270164107
2024-08-29 23:58:38.166001 Epoch 40, Training loss 0.26012524117709723
2024-08-29 23:58:52.181403 Epoch 50, Training loss 0.24278048478114378
2024-08-29 23:59:06.990764 Epoch 60, Training loss 0.2244673086104879
2024-08-29 23:59:24.056534 Epoch 70, Training loss 0.20878569701105165
2024-08-29 23:59:40.142930 Epoch 80, Training loss 0.19548199534606023
2024-08-29 23:59:55.778227 Epoch 90, Training loss 0.17811112497357806
2024-08-30 00:00:11.404136 Epoch 100, Training loss 0.16377417966248883


In [31]:
def validate(model, train_loader, val_loader): 
    for name, loader in [("train", train_loader), ("val", val_loader)]: 
        correct = 0 
        total = 0 
        with torch.no_grad(): 
            for imgs, labels in loader: 
                outputs = model(imgs) 
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0] 
                correct += int((predicted == labels).sum()) 
        print("Accuracy {}: {:.2f}".format(name , correct / total)) 

In [32]:
validate(model, train_loader, val_loader)

Accuracy train: 0.94
Accuracy val: 0.89


In [33]:
torch.save(model.state_dict(), datapath + 'birds_vs_airplanes.pt') 
#保存模型的参数即 2 个卷积模块和 2 个线性模块的权重和偏置